# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [ ]:
#langchain library
#!pip install langchain==0.0.350
#LLM
#!pip install accelerate==0.25.0
#!pip install transformers==4.36.2
#!pip install bitsandbytes==0.41.2
#Text Embedding
#!pip install sentence-transformers==2.2.2
#!pip install InstructorEmbedding==1.0.1
#vectorstore
#!pip install pymupdf==1.23.8
#!pip install faiss-gpu==1.7.2
#!pip install faiss-cpu==1.7.4

In [1]:
import os
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [2]:
from langchain import PromptTemplate

prompt_template = """
    HI ,I am AIT LILbot. I am your study program bot .If you are looking for courses offered in AIT and which school offers which courses you can ask me, I am always ready to help you ^-^!
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format 
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template='HI ,I am AIT LILbot. I am your study program bot .If you are looking for courses offered in AIT and which school offers which courses you can ask me, I am always ready to help you ^-^!\n    {context}\n    Question: {question}\n    Answer:')

In [3]:
PROMPT.format(
    context = "All programs information are given below.",
    question = "What are the programs offered?"
)

'HI ,I am AIT LILbot. I am your study program bot .If you are looking for courses offered in AIT and which school offers which courses you can ask me, I am always ready to help you ^-^!\n    All programs information are given below.\n    Question: What are the programs offered?\n    Answer:'

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [ ]:
try:
    import pwd
except ImportError:
    # Handle the ImportError (e.g., provide an alternative implementation or raise a more informative error)
    print("The pwd module is not available on this platform.")


In [4]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [5]:
documents

[Document(page_content='3/21/24, 7:37 PM\nAcademic Programs - Asian Institute of Technology\nhttps://ait.ac.th/academics/programs/\n1/9\nHome (h�ps://ait.ac.th) > Academics (h�ps://ait.ac.th… > Academic Programs\nAcademic Programs\nWe oﬀer various degree programs. You\'ll sample courses in a wide range of\nsubjects before immersing yourself in one of these focused areas.\nIn this sec�on\nShow ﬁlters\n (/)\nWe use cookies on our website to give you the most relevant experience by remembering your preferences and repeat visits. By clicking “Accept\nAll”, you consent to the use of ALL the cookies. However, you may visit "Cookie Settings" to provide a controlled consent.\nCookie Settings\nAccept All\n', metadata={'source': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'file_path': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'page': 0, 'total_pages': 9, 'form

In [6]:
len(documents)

9

In [7]:
documents[1]

Document(page_content='3/21/24, 7:37 PM\nAcademic Programs - Asian Institute of Technology\nhttps://ait.ac.th/academics/programs/\n2/9\nAll Programs\nAgri-Business Management\n(ABM)\nAgricultural Systems And\nEngineering (ASE)\nAquaculture and Aqua�c\nResources Management\n(AARM)\nBio-Nano Materials Science\nand Engineering (BNMSE)\nClimate Change and\nSustainable Development\n(CCSD)\nComputer Science (CS)\n (/)\nWe use cookies on our website to give you the most relevant experience by remembering your preferences and repeat visits. By clicking “Accept\nAll”, you consent to the use of ALL the cookies. However, you may visit "Cookie Settings" to provide a controlled consent.\nCookie Settings\nAccept All\n', metadata={'source': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'file_path': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'page': 1, 'total_pages': 9

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [9]:
doc[1]

Document(page_content='3/21/24, 7:37 PM\nAcademic Programs - Asian Institute of Technology\nhttps://ait.ac.th/academics/programs/\n2/9\nAll Programs\nAgri-Business Management\n(ABM)\nAgricultural Systems And\nEngineering (ASE)\nAquaculture and Aqua�c\nResources Management\n(AARM)\nBio-Nano Materials Science\nand Engineering (BNMSE)\nClimate Change and\nSustainable Development\n(CCSD)\nComputer Science (CS)\n (/)\nWe use cookies on our website to give you the most relevant experience by remembering your preferences and repeat visits. By clicking “Accept\nAll”, you consent to the use of ALL the cookies. However, you may visit "Cookie Settings" to provide a controlled consent.\nCookie Settings\nAccept All', metadata={'source': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'file_path': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'page': 1, 'total_pages': 9, 

In [10]:
len(doc)

13

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [11]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

c:\Users\Haneesha\AppData\Local\Programs\Python\Python311\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
c:\Users\Haneesha\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


load INSTRUCTOR_Transformer


c:\Users\Haneesha\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Haneesha\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [12]:
#locate vectorstore
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [13]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [14]:
vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp', #default index
    allow_dangerous_deserialization = True
)


In [15]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   

ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and no that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

In [16]:
#ready to use
retriever = vectordb.as_retriever()

In [17]:
retriever.get_relevant_documents("What are the course structure in Agri-Business Management?")

[Document(page_content='3/21/24, 7:37 PM\nAcademic Programs - Asian Institute of Technology\nhttps://ait.ac.th/academics/programs/\n2/9\nAll Programs\nAgri-Business Management\n(ABM)\nAgricultural Systems And\nEngineering (ASE)\nAquaculture and Aqua�c\nResources Management\n(AARM)\nBio-Nano Materials Science\nand Engineering (BNMSE)\nClimate Change and\nSustainable Development\n(CCSD)\nComputer Science (CS)\n (/)\nWe use cookies on our website to give you the most relevant experience by remembering your preferences and repeat visits. By clicking “Accept\nAll”, you consent to the use of ALL the cookies. However, you may visit "Cookie Settings" to provide a controlled consent.\nCookie Settings\nAccept All', metadata={'source': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'file_path': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'page': 1, 'total_pages': 9,

In [18]:
retriever.get_relevant_documents("Who are the faculty in Data Science and AI?")

[Document(page_content='3/21/24, 7:37 PM\nAcademic Programs - Asian Institute of Technology\nhttps://ait.ac.th/academics/programs/\n9/9\n©2022 Asian Institute of Technology. All Rights Reserved. - Designed by Outsourcify (https://outsourcify.net)\n (/)\nWe use cookies on our website to give you the most relevant experience by remembering your preferences and repeat visits. By clicking “Accept\nAll”, you consent to the use of ALL the cookies. However, you may visit "Cookie Settings" to provide a controlled consent.\nCookie Settings\nAccept All', metadata={'source': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'file_path': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'page': 8, 'total_pages': 9, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, lik

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [19]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [20]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [21]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [22]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [24]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [34]:
%cd ./models
!git clone 'https://huggingface.co/lmsys/fastchat-t5-3b-v1.0'

[WinError 2] The system cannot find the file specified: './models'
c:\Users\Haneesha\OneDrive\Desktop\NLP\A7


'git' is not recognized as an internal or external command,
operable program or batch file.


In [38]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("lmsys/fastchat-t5-3b-v1.0")
model = AutoModelForSeq2SeqLM.from_pretrained("lmsys/fastchat-t5-3b-v1.0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/6.71G [00:00<?, ?B/s]

c:\Users\Haneesha\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Haneesha\.cache\huggingface\hub\models--lmsys--fastchat-t5-3b-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [44]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain import HuggingFacePipeline

model_id = "lmsys/fastchat-t5-3b-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    model_kwargs={
        "temperature": 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline=pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [45]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [46]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [47]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [48]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

c:\Users\Haneesha\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  are  the  main  differences  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [49]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template='HI ,I am AIT LILbot. I am your study program bot .If you are looking for courses offered in AIT and which school offers which courses you can ask me, I am always ready to help you ^-^!\n    {context}\n    Question: {question}\n    Answer:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x000002B2E71A9B90>)), document_variable_name='context')

In [50]:
query = "How many Credits Does a course work have in Data Science and AI"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
HI ,I am AIT LILbot. I am your study program bot .If you are looking for courses offered in AIT and which school offers which courses you can ask me, I am always ready to help you ^-^!
    3/21/24, 7:37 PM
Academic Programs - Asian Institute of Technology
https://ait.ac.th/academics/programs/
9/9
©2022 Asian Institute of Technology. All Rights Reserved. - Designed by Outsourcify (https://outsourcify.net)
 (/)
We use cookies on our website to give you the most relevant experience by remembering your preferences and repeat visits. By clicking “Accept
All”, you consent to the use of ALL the cookies. However, you may visit "Cookie Settings" to provide a controlled consent.
Cookie Settings
Accept All

3/21/24, 7:37 PM
Academic Programs - Asian Institute of Technology
https://ait.ac.th/academics/programs/
2/9
All Programs
Agri-Business Management
(ABM)
Agricultural Systems And
Engineerin

{'input_documents': [Document(page_content='3/21/24, 7:37 PM\nAcademic Programs - Asian Institute of Technology\nhttps://ait.ac.th/academics/programs/\n9/9\n©2022 Asian Institute of Technology. All Rights Reserved. - Designed by Outsourcify (https://outsourcify.net)\n (/)\nWe use cookies on our website to give you the most relevant experience by remembering your preferences and repeat visits. By clicking “Accept\nAll”, you consent to the use of ALL the cookies. However, you may visit "Cookie Settings" to provide a controlled consent.\nCookie Settings\nAccept All', metadata={'source': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'file_path': 'C:\\Users\\Haneesha\\OneDrive\\文件\\Boooooookkkkkkssss\\Academic Programs - Asian Institute of Technology.pdf', 'page': 8, 'total_pages': 9, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKi

In [51]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template='HI ,I am AIT LILbot. I am your study program bot .If you are looking for courses offered in AIT and which school offers which courses you can ask me, I am always ready to help you ^-^!\n    {context}\n    Question: {question}\n    Answer:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x000002B2E71A9B90>)), document_variable_name='context'), question_generator=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\

## 5. Chatbot

In [52]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
HI ,I am AIT LILbot. I am your study program bot .If you are looking for courses offered in AIT and which school offers which courses you can ask me, I am always ready to help you ^-^!
    3/21/24, 7:37 PM
Academic Programs - Asian Institute of Technology
https://ait.ac.th/academics/programs/
9/9
©2022 Asian Institute of Technology. All Rights Reserved. - Designed by Outsourcify (https://outsourcify.net)
 (/)
We use cookies on our website to give you the most relevant experience by remembering your preferences and repeat visits. By clicking “Accept
All”, you consent to the use of ALL the cookies. However, you may visit "Cookie Settings" to provide a controlled consent.
Cookie Settings
Accept All

3/21/24, 7:37 PM
Academic Programs - Asian Institute of Technology
https://ait.ac.th/academics/programs/
5/9
Dual Degree Program with

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': '<pad>  I  am  an  AI  bot  created  by  Asian  Institute  of  Technology  (AIT)  to  provide  information  about  academic  programs  offered  by  the  institute.  I  am  designed  to  assist  users  in  finding  the  courses  they  are  interested  in  and  which  schools  offer  those  courses.\n',
 'source_documents': [Document(page_content='3/21/24, 7:37 PM\nAcademic Programs - Asian Institute of Technology\nhttps://ait.ac.th/academics/programs/\n9/9\n©2022 Asian Institute of Technology. All Rights Reserved. - Designed by Outsourcify (https://outsourcify.net)\n (/)\nWe use cookies on our website to give you the most relevant experience by remembering your preferences and repeat visits. By clicking “Accept\nAll”, you consent to the use of ALL the cookies. However, you may visit "Cookie Settings" to provide a controlled consent.\nCookie Settings\nAccept All', metadata={'source': 'C:\\Users\\Haneesha\\OneDrive\\文

In [53]:
prompt_question = "What are the programs offered in August semester?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  an  AI  bot  created  by  Asian  Institute  of  Technology  (AIT)  to  provide  information  about  academic  programs  offered  by  the  institute.  I  am  designed  to  assist  users  in  finding  the  courses  they  are  interested  in  and  which  schools  offer  those  courses.\n')]
Follow Up Input: What are the programs offered in August semester?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
HI ,I am AIT LILbot. I am your study program bot .If you are looking for courses offered in AIT and which school offers which courses you c

{'question': 'What are the programs offered in August semester?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  an  AI  bot  created  by  Asian  Institute  of  Technology  (AIT)  to  provide  information  about  academic  programs  offered  by  the  institute.  I  am  designed  to  assist  users  in  finding  the  courses  they  are  interested  in  and  which  schools  offer  those  courses.\n')],
 'answer': '<pad> < pad>\n The  August  semester  at  AIT  offers  the  following  programs:\n *  Bachelor  of  Science  (B.S.)  in  Computer  Science\n *  Bachelor  of  Science  (B.S.)  in  Information  Technology\n *  Bachelor  of  Science  (B.S.)  in  Computer  Engineering\n *  Bachelor  of  Science  (B.S.)  in  Information  Technology  Management\n *  Bachelor  of  Science  (B.S.)  in  Computer  Science\n *  Bachelor  of  Science  (B.S.)  in  Information  Technology  Management\n *  Bachelor  of  Science  (B.S.)  in  Computer  Engin

**Issues faced during modeling**
I haven't used bits and bytes because of the cpu issue,as per the model it has given relevant data along with file details and website links